# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), July 16, 2019**

In [1]:
using DataFrames

## Handling missing values

A singelton type `Missing` allows us to deal with missing values.

In [2]:
missing, typeof(missing)

(missing, Missing)

Arrays automatically create an appropriate union type.

In [3]:
x = [1, 2, missing, 3]

4-element Array{Union{Missing, Int64},1}:
 1       
 2       
  missing
 3       

`ismissing` checks if passed value is missing.

In [4]:
ismissing(1), ismissing(missing), ismissing(x), ismissing.(x)

(false, true, false, Bool[false, false, true, false])

We can extract the type combined with Missing from a `Union` via `Missings.T`

(This is useful for arrays!)

In [5]:
eltype(x), Missings.T(eltype(x))

(Union{Missing, Int64}, Int64)

`missing` comparisons produce `missing`.

In [6]:
missing == missing, missing != missing, missing < missing

(missing, missing, missing)

This is also true when `missing`s are compared with values of other types.

In [7]:
1 == missing, 1 != missing, 1 < missing

(missing, missing, missing)

`isequal`, `isless`, and `===` produce results of type `Bool`. Notice that `missing` is considered greater than any numeric value.

In [8]:
isequal(missing, missing), missing === missing, isequal(1, missing), isless(1, missing)

(true, true, false, true)

In the next few examples, we see that many (not all) functions handle `missing`.

In [9]:
map(x -> x(missing), [sin, cos, zero, sqrt]) # part 1

4-element Array{Missing,1}:
 missing
 missing
 missing
 missing

In [10]:
map(x -> x(missing, 1), [+, - , *, /, div]) # part 2 

5-element Array{Missing,1}:
 missing
 missing
 missing
 missing
 missing

In [11]:
using Statistics # needed for mean
map(x -> x([1,2,missing]), [minimum, maximum, extrema, mean, float]) # part 3

5-element Array{Any,1}:
 missing                                   
 missing                                   
 (missing, missing)                        
 missing                                   
 Union{Missing, Float64}[1.0, 2.0, missing]

`skipmissing` returns iterator skipping missing values. We can use `collect` and `skipmissing` to create an array that excludes these missing values.

In [12]:
collect(skipmissing([1, missing, 2, missing]))

2-element Array{Int64,1}:
 1
 2

Here we use `replace` to create a new array that replaces all missing values with some value (`NaN` in this case).

In [13]:
replace([1.0, missing, 2.0, missing], missing=>NaN)

4-element Array{Float64,1}:
   1.0
 NaN  
   2.0
 NaN  

Another way to do this:

In [14]:
coalesce.([1.0, missing, 2.0, missing], NaN)

4-element Array{Float64,1}:
   1.0
 NaN  
   2.0
 NaN  

You can also use `recode` if you have a default output value.

In [15]:
recode([1.0, missing, 2.0, missing], false, missing=>true)

4-element Array{Bool,1}:
 false
  true
 false
  true

There are also `replace!` and `recode!` functions that work in place.

Here is an example how you can to missing inputation in a data frame.

In [16]:
df = DataFrame(a=[1,2,missing], b=["a", "b", missing])

,a,b
,Int64⍰,String⍰
1,1,a
2,2,b
3,missing,missing


we change `df.a` vector in place.

In [17]:
replace!(df.a, missing=>100)

3-element Array{Union{Missing, Int64},1}:
   1
   2
 100

Now we overwrite `df.b` with a new vector, because the replacement type is different than what `eltype(df.b)` accepts.

In [18]:
df.b = coalesce.(df.b, 100)

3-element Array{Any,1}:
    "a"
    "b"
 100   

In [19]:
df

,a,b
,Int64⍰,Any
1,1,a
2,2,b
3,100,100


You can use `unique` or `levels` to get unique values with or without missings, respectively.

In [20]:
unique([1, missing, 2, missing]), levels([1, missing, 2, missing])

(Union{Missing, Int64}[1, missing, 2], [1, 2])

In this next example, we convert `x` to `y` with `allowmissing`, where `y` has a type that accepts missings.

In [21]:
x = [1,2,3]
y = allowmissing(x)

3-element Array{Union{Missing, Int64},1}:
 1
 2
 3

Then, we convert back with `disallowmissing`. This would fail if `y` contained missing values!

In [22]:
z = disallowmissing(y)
x,y,z

([1, 2, 3], Union{Missing, Int64}[1, 2, 3], [1, 2, 3])

In this next example, we show that the type of each column in `x` is initially `Int64`. After using `allowmissing!` to accept missing values in columns 1 and 3, the types of those columns become `Union{Int64,Missing}`.

In [23]:
x = DataFrame(rand(Int, 2,3))
println("Before: ", eltypes(x))
allowmissing!(x, 1) # make first column accept missings
allowmissing!(x, :x3) # make :x3 column accept missings
println("After: ", eltypes(x))

Before: DataType[Int64, Int64, Int64]
After: Type[Union{Missing, Int64}, Int64, Union{Missing, Int64}]


In this next example, we'll use `completecases` to find all the rows of a `DataFrame` that have complete data.

In [24]:
x = DataFrame(A=[1, missing, 3, 4], B=["A", "B", missing, "C"])

,A,B
,Int64⍰,String⍰
1,1,A
2,missing,B
3,3,missing
4,4,C


In [25]:
println("Complete cases:\n", completecases(x))

Complete cases:
Bool[true, false, false, true]


We can use `dropmissing` or `dropmissing!` to remove the rows with incomplete data from a `DataFrame` and either create a new `DataFrame` or mutate the original in-place.

In [26]:
y = dropmissing(x)
dropmissing!(x)

,A,B
,Int64,String
1,1,A
2,4,C


In [27]:
x

,A,B
,Int64,String
1,1,A
2,4,C


In [28]:
y

,A,B
,Int64,String
1,1,A
2,4,C


When we call `describe` on a `DataFrame` with dropped missing values, the columns do not allow missing values any more by default.

In [29]:
describe(x)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,A,2.5,1,2.5,4,,,Int64
2,B,,A,,C,2,,String


Alternatively you can pass `disallowmissing` keyword argument to `dropmissing` and `dropmissing!`

In [30]:
x = DataFrame(A=[1, missing, 3, 4], B=["A", "B", missing, "C"])

,A,B
,Int64⍰,String⍰
1,1,A
2,missing,B
3,3,missing
4,4,C


In [31]:
dropmissing!(x, disallowmissing=false)

,A,B
,Int64⍰,String⍰
1,1,A
2,4,C


### Making functions `missing`-aware

If we have a function that does not handle `missing` values we can wrap it using `passmissing` function so that if any of its positional arguments is missing we will get a `missing` value in return. In the example below we change how `string` function behaves:

In [32]:
string(missing)

"missing"

In [33]:
string(missing, " ", missing)

"missing missing"

In [34]:
string(1,2,3)

"123"

In [35]:
lift_string = passmissing(string)

(::Missings.PassMissing{typeof(string)}) (generic function with 2 methods)

In [36]:
lift_string(missing)

missing

In [37]:
lift_string(missing, " ", missing)

missing

In [38]:
lift_string(1,2,3)

"123"